In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: Miniproject, id: 10pecghBomFnoaqcW3BL54CGqR5DX8BqjGoCFRiAPsEM
title: Doghumannot, id: 1UiBgnGZGk4r4hLzpF-RHTBrG9J5dC8Fm
title: ptsp assignment, id: 11iaS6TGgEjJ2kLxggY4UKcwo6RiNR2u3
title: my lines, id: 1EdrwBKZxm9qp3_CoZhxNJFcbAqLgCTKv8-Cv6o6Lf4Q
title: my lines, id: 1DSJEEF4U8t_fsikX05a_IqTe7aWd4y7qUYCNWTStQRM
title: my lines, id: 1RTi_8wWNhcAseJkNV5qkz0B3-H7xJTbguogmcnMYlNs
title: my lines, id: 1JOC1PYH1tCJKK3HOlGpAei6Q-AhM0JE47j9gr2tjpBs
title: my lines, id: 1b2UIPXVbycLuA6PdRtUm2e-wIT-RsF-B5KZDirvVgRU
title: oops assignment 2, id: 1-4J_KuVGQRFbBd_bb4u-yeyqrnFvCUyX
title: Sup, id: 1Ub5vkzeSPxQpJIR_CLrgFYIHZTsF67Ne
title: Copy of 109,099 SSS miniproject report, id: 1D77KmyAjCeV6klQDkgsEf8fpYrhW_cY3rVBujziTi9E
title: 109,099 SSS miniproject report.pdf, id: 1xB0qvt4ipVQcybAtWQntOVaI9ro4Nsy6
title: 109,099 SSS miniproject report.docx, id: 1C4mVaBJIvfR1iwDrJ3qwUUqzZJq7hIhz
title: 109suprajaw2, id: 1tBbhseuXutWWsOeCVk0W3MPzO84R8V-A
title: 109supraja.pdf, id: 1tAwM1NQZF7q7BiExHTs3c

In [0]:
downloaded = drive.CreateFile({'id': '16qBB5M1R81wW_bEtCs4eJg1SVHsQn9q4'})
downloaded.GetContentFile('cnndh.zip')

In [6]:
!unzip cnndh.zip

unzip:  cannot find or open cnndh.zip, cnndh.zip.zip or cnndh.zip.ZIP.


In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) 

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [0]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 

In [0]:
CLASSES = 3
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [0]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'Dog_Human/train'
TEST_DIR=r'Dog_Human/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
print(train_generator.class_indices)
print(validation_generator.class_indices)

Found 19810 images belonging to 3 classes.
Found 3504 images belonging to 3 classes.
{'dog': 0, 'human': 1, 'nothd': 2}
{'dog': 0, 'human': 1, 'nothd': 2}


In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
EPOCHS = 2
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS=64
model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS
    )

Epoch 1/2
220/320 [===================>..........] - ETA: 6:15 - loss: 0.1303 - accuracy: 0.9546

In [0]:
from tensorflow.python.keras.preprocessing import image

In [0]:
img = image.load_img(r'Dog_Human/test/dog/1003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = pred.argmax(axis=-1)
y_classes

In [0]:
img = image.load_img(r'Dog_Human/test/human/Sharon_Osbourne_0003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = np.argmax(pred)
y_classes

In [0]:
from keras.models import model_from_json
model_json = model.to_json()

with open("dog_cnn.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("dog_cnn.h5")

In [0]:
train_generator.class_indices

In [0]:
df.loc[df['category']>100, 'image_name']